In [1]:
# import libraries
import os
from dotenv import load_dotenv
from base64 import b64encode
import requests
import pandas as pd

In [2]:
# get client_id and client_secret from environment variables
load_dotenv()
client_id = os.environ.get("CLIENT_ID")
client_secret = os.environ.get("CLIENT_SECRET")

In [3]:
## get authorization token
## docs - https://developer.spotify.com/documentation/web-api/tutorials/client-credentials-flow
# prepare authorization string
auth_string = f"{client_id}:{client_secret}"
# encode to bytes
auth_bytes = auth_string.encode("utf-8")
# encode to base64
auth_base64 = b64encode(auth_bytes).decode()

# api url
url = "https://accounts.spotify.com/api/token"
# authorization options
headers = {
    "Authorization": f"Basic {auth_base64}",
    "Content-Type": "application/x-www-form-urlencoded"
}
form = {"grant_type": "client_credentials"}

# post authorization request
response = requests.post(url=url, headers=headers, data=form)
# extract token from response
token = response.json()["access_token"]

In [4]:
## get playlist data
## docs - https://developer.spotify.com/documentation/web-api/reference/get-playlist
# playlist id
playlist_id = "37i9dQZEVXbMnz8KIWsvf9"

# api url
url = f"https://api.spotify.com/v1/playlists/{playlist_id}"
# authorization options
headers = {"Authorization": f"Bearer {token}"}

# get response
response = requests.get(url=url, headers=headers)
# convert to dictionary
result = response.json()

In [5]:
# inspect album data
print(result["tracks"]["items"][0]["track"]["album"]["id"])
print(result["tracks"]["items"][0]["track"]["album"]["name"])
print(result["tracks"]["items"][0]["track"]["album"]["release_date"])
print(result["tracks"]["items"][0]["track"]["album"]["total_tracks"])

3jeQDa9OFZ6GndLindHx3k
Take Two
2023-06-09
1


In [6]:
# inspect track data
print(result["tracks"]["items"][0]["track"]["id"])
print(result["tracks"]["items"][0]["track"]["name"])
print(result["tracks"]["items"][0]["track"]["duration_ms"])
print(result["tracks"]["items"][0]["track"]["popularity"])
print(result["tracks"]["items"][0]["track"]["album"]["id"])
print(result["tracks"]["items"][0]["track"]["artists"][0]["id"])

5IAESfJjmOYu7cHyX557kz
Take Two
229953
93
3jeQDa9OFZ6GndLindHx3k
3Nrfpe0tUJi4K4DXYWgMUX


In [7]:
# inspect artist data
print(result["tracks"]["items"][0]["track"]["artists"][0]["id"])
print(result["tracks"]["items"][0]["track"]["artists"][0]["name"])

3Nrfpe0tUJi4K4DXYWgMUX
BTS


In [8]:
## extract album data
# empty album list
album_list = []

# for loop to extract data
for album in result["tracks"]["items"]:
    # extract album data
    album_dict = {
        "id": album["track"]["album"]["id"],
        "name": album["track"]["album"]["name"],
        "release_date": album["track"]["album"]["release_date"],
        "total_tracks": album["track"]["album"]["total_tracks"]
    }
    # if extracted data not in list then append to list
    if album_dict not in album_list:
        album_list.append(album_dict)

In [9]:
## extract artist data
# empty artist list
artist_list = []

# for loop to extract data
for artists in result["tracks"]["items"]:
    # extract artist data
    for artist in artists["track"]["artists"]:
        artist_dict = {
            "artist_id": artist["id"],
            "artist_name": artist["name"]
        }
        # if extracted data not in list then append to list
        if artist_dict not in artist_list:
            artist_list.append(artist_dict)

In [10]:
# define convert milliseconds to minutes function
def milli_to_min(milliseconds):
    return round(milliseconds / 60000, 2)

## extract track data
# empty track list
track_list = []

# for loop to extract data
for track in result["tracks"]["items"]:
    # extract track data
    track_dict = {
        "track_id": track["track"]["id"],
        "track_name": track["track"]["name"],
        "duration": milli_to_min(track["track"]["duration_ms"]),
        "popularity": track["track"]["popularity"],
        "album_id": track["track"]["album"]["id"],
        "artist_id": track["track"]["artists"][0]["id"]
    }
    # if extracted data not in list then append to list
    if track_dict not in track_list:
        track_list.append(track_dict)

In [11]:
# view album data as dataframe
pd.DataFrame(album_list).head()

,id,name,release_date,total_tracks
0,3jeQDa9OFZ6GndLindHx3k,Take Two,2023-06-09,1
1,66OYt73mqan1hWa78BhfPd,LALISA,2021-09-10,2
2,5Jk4Eg7pxYhDrWJCVVzmMt,CHARLIE,2022-10-06,12
3,4xc3Lc9yASZgEJGH7acWMB,FACE,2023-03-24,6
4,2H5nzViLpbX8JzY1cQMVza,"Angel Pt. 1 (feat. Jimin of BTS, JVKE & Muni L...",2023-05-18,2


In [12]:
# view artist data as dataframe
pd.DataFrame(artist_list).head()

,artist_id,artist_name
0,3Nrfpe0tUJi4K4DXYWgMUX,BTS
1,5L1lO4eRHmJ7a0Q6csE5cT,LISA
2,6VuMaDnrHyPL1p4EHjYLi7,Charlie Puth
3,6HaGTQPmzraVmaVxvz6EUc,Jung Kook
4,1oSPZhvZMIrWW5I41kPkkY,Jimin


In [13]:
# view track as dataframe
pd.DataFrame(track_list).head()

,track_id,track_name,duration,popularity,album_id,artist_id
0,5IAESfJjmOYu7cHyX557kz,Take Two,3.83,93,3jeQDa9OFZ6GndLindHx3k,3Nrfpe0tUJi4K4DXYWgMUX
1,7hU3IHwjX150XLoTVmjD0q,MONEY,2.80,82,66OYt73mqan1hWa78BhfPd,5L1lO4eRHmJ7a0Q6csE5cT
2,5Odq8ohlgIbQKMZivbWkEo,Left and Right (Feat. Jung Kook of BTS),2.57,86,5Jk4Eg7pxYhDrWJCVVzmMt,6VuMaDnrHyPL1p4EHjYLi7
3,3Ua0m0YmEjrMi9XErKcNiR,Like Crazy,3.54,94,4xc3Lc9yASZgEJGH7acWMB,1oSPZhvZMIrWW5I41kPkkY
4,2yjeXq5ahx6UUg2rHaUe6X,"Angel Pt. 2 (feat. Jimin of BTS, Charlie Puth ...",2.94,68,2H5nzViLpbX8JzY1cQMVza,164Uj4eKjl6zTBKfJLFKKK
